## 1. `PromptTemplate`

### Description:
- `PromptTemplate` is the most basic template class.
- It is used to generate a string prompt by filling variables into a pre-defined template.
- Suitable for models that accept plain text as input (like OpenAI's `text-davinci` or `GPT-3` variants).

### Key Use-Cases:
- Prompting text-only models.
- Structuring basic question-answering prompts.
- Used in classical RAG pipelines where the prompt is just a single formatted string.

In [1]:
# ===================== INSTALL DEPENDENCIES =====================
!pip install -q langchain sentence-transformers faiss-cpu pypdf groq langchain-community langchain-groq nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [2]:
# ===================== IMPORTS =====================
import os
import torch
import re
import nltk
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from sentence_transformers.cross_encoder import CrossEncoder
from IPython.display import display, Markdown

In [3]:
# ===================== DOWNLOAD NLTK RESOURCES =====================
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
# ===================== LOAD & SPLIT PDF =====================
loader = PyPDFLoader("/content/solid-python.pdf")
documents = loader.load_and_split()

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)
print(f"Total Chunks Created: {len(docs)}")

Total Chunks Created: 53


In [5]:
# ===================== EMBEDDINGS + VECTORSTORE =====================
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

/tmp/ipython-input-5-1768769302.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# ===================== RETRIEVER WITH MMR =====================
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ae9b3742290>, search_type='mmr', search_kwargs={'k': 5})

In [22]:
# ===================== DEFINE LLM =====================
from google.colab import userdata
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=userdata.get('GROQ_API_KEY')
)

In [23]:
# ===================== DEFINE PROMPT =====================
prompt_template = PromptTemplate.from_template(
    "Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
)

In [24]:
# ===================== RERANKER INITIALIZATION =====================
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

In [25]:
# ===================== ASK A QUESTION =====================
question = "What is Solid Princiles?"

retrieved_docs = retriever.get_relevant_documents(question)

In [26]:
# Display retrieved chunks (before reranking)
print("\n Top K Retrieved Chunks (Before Reranking):")
for i, doc in enumerate(retrieved_docs):
    page = doc.metadata.get("page", "Unknown")
    print(f"\n--- Chunk {i+1} ---")
    print(f"Page: {page}")
    print(f"Content:\n{doc.page_content[:300]}...")


 Top K Retrieved Chunks (Before Reranking):

--- Chunk 1 ---
Page: 41
Content:
car_wash code example
https://github.com/aleasoluciones/car_wash
SOLID definition (at wikipedia)
http://en.wikipedia.org/wiki/SOLID_(object-oriented_design)
Getting a SOLID start (Uncle Bob)
http://butunclebob.com/ArticleS.UncleBob.PrinciplesOfOod
Video SOLID Object Oriented Design (Sandi Metz)
http...

--- Chunk 2 ---
Page: 32
Content:
Duck Typing Approved!!!...

--- Chunk 3 ---
Page: 1
Content:
Alea Soluciones 
Bifer Team...

--- Chunk 4 ---
Page: 3
Content:
Usual OO Systems
Rigid
Fragile
Immobile
Viscous...

--- Chunk 5 ---
Page: 40
Content:
SOLID Motivational Posters, by Derick Bailey
http://lostechies.com/derickbailey/2009/02/11/solid-development-principles-in-motivational-pictures/...


In [28]:
# ===================== RERANK CHUNKS =====================
pairs = [[question, doc.page_content] for doc in retrieved_docs]
scores = reranker.predict(pairs)

scored_docs = list(zip(retrieved_docs, scores))
sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)

In [29]:
# ===================== FINAL ANSWERS =====================
context_before = "\n\n".join([doc.page_content for doc in retrieved_docs[:3]])
prompt_before = prompt_template.format(context=context_before, question=question)
answer_before = llm.invoke(prompt_before)

top_reranked_docs = [doc for doc, _ in sorted_docs[:3]]
context_after = "\n\n".join([doc.page_content for doc in top_reranked_docs])
prompt_after = prompt_template.format(context=context_after, question=question)
answer_after = llm.invoke(prompt_after)

In [30]:
# ===================== DISPLAY RESULTS =====================
display(Markdown("### Final Answer (Before Reranking):"))
display(Markdown(answer_before.content))

display(Markdown("### Final Answer (After Reranking):"))
display(Markdown(answer_after.content))

### Final Answer (Before Reranking):

SOLID principles are a set of design principles in object-oriented programming (OOP) that aim to promote cleaner, more robust, and maintainable code. The acronym SOLID stands for:

1. **S** - Single Responsibility Principle (SRP): A class should have only one reason to change, meaning it should have a single responsibility or purpose.
2. **O** - Open/Closed Principle (OCP): A class should be open for extension but closed for modification, meaning you should be able to add new functionality without changing the existing code.
3. **L** - Liskov Substitution Principle (LSP): Derived classes should be substitutable for their base classes, meaning any code that uses a base class should be able to work with a derived class without knowing the difference.
4. **I** - Interface Segregation Principle (ISP): A client should not be forced to depend on interfaces it does not use, meaning instead of having a large, fat interface, you should have multiple smaller, more focused interfaces.
5. **D** - Dependency Inversion Principle (DIP): High-level modules should not depend on low-level modules, but both should depend on abstractions, meaning instead of depending on concrete implementations, you should depend on abstract interfaces.

These principles were first introduced by Robert C. Martin, also known as "Uncle Bob," and are widely accepted as a foundation for good object-oriented design. They help developers create more modular, flexible, and maintainable software systems.

### Final Answer (After Reranking):

The SOLID principles are a set of five design principles in object-oriented programming (OOP) that aim to promote simpler, more robust, and updatable code for software development in object-oriented languages. Each letter in SOLID represents a principle for development:

1. **S** - Single Responsibility Principle (SRP): A class should have only one reason to change, meaning it should have only one job or responsibility.
2. **O** - Open/Closed Principle (OCP): A class should be open for extension but closed for modification, meaning you should be able to add new functionality without changing the existing code.
3. **L** - Liskov Substitution Principle (LSP): Derived classes should be substitutable for their base classes, meaning any code that uses a base class should be able to work with a derived class without knowing the difference.
4. **I** - Interface Segregation Principle (ISP): A client should not be forced to depend on interfaces it does not use, meaning instead of having a large, fat interface, break it up into smaller, more specialized interfaces.
5. **D** - Dependency Inversion Principle (DIP): High-level modules should not depend on low-level modules, but both should depend on abstractions. Additionally, abstractions should not depend on details, but details should depend on abstractions.

These principles were introduced to help developers design more maintainable, flexible, and scalable software systems, avoiding the common problems found in usual OO systems that are rigid, fragile, immobile, or viscous.

In [31]:
# ===================== PROMPT QUALITY METRICS =====================
# === Ground Truth Answer ===
ground_truth = "The main objective of the document is to teach Python programming with solid design principles."

# === Model Answers ===
model_answer_before = answer_before.content.strip()
model_answer_after = answer_after.content.strip()

In [35]:
# === Metric Functions ===
def preprocess(text):
    return re.sub(r'[^\w\s]', '', text.lower()).split()

def exact_match(prediction, ground_truth):
    """Strict equality check (EM)"""
    return int(prediction.strip().lower() == ground_truth.strip().lower())

def f1_token_overlap(prediction, ground_truth):
    """F1 score based on overlapping tokens"""
    pred_tokens = preprocess(prediction)
    gt_tokens = preprocess(ground_truth)
    common = set(pred_tokens) & set(gt_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    return 2 * (precision * recall) / (precision + recall)

def token_accuracy(prediction, ground_truth):
    """Overlap-based Accuracy (1 if any token matches)"""
    pred_tokens = preprocess(prediction)
    gt_tokens = preprocess(ground_truth)
    return int(len(set(pred_tokens) & set(gt_tokens)) > 0)

def pass_at_k(generations, ground_truth):
    """Checks if any of the k generations match exactly"""
    return int(any(exact_match(g, ground_truth) for g in generations))

def compute_meteor(prediction, ground_truth):
    """Semantic metric accounting for synonyms and paraphrasing"""
    # Preprocess the prediction and ground_truth into tokens for METEOR calculation
    prediction_tokens = preprocess(prediction)
    ground_truth_tokens = preprocess(ground_truth)
    return round(nltk.translate.meteor_score.single_meteor_score(ground_truth_tokens, prediction_tokens), 4)

In [36]:
# === Run Evaluations ===
generated_attempts = [
    "To teach Python programming with good practices.",
    "The book aims to teach Python using solid design principles.",
    model_answer_after
]

In [37]:
def evaluate_all(prediction, label, attempts=None):
    return {
        "Exact Match (EM)": exact_match(prediction, label),
        "F1 (Token Overlap)": round(f1_token_overlap(prediction, label), 4),
        "Accuracy (Overlap-based)": token_accuracy(prediction, label),
        "Pass@3": pass_at_k(attempts if attempts else [prediction], label),
        "METEOR": compute_meteor(prediction, label)
    }

metrics_before = evaluate_all(model_answer_before, ground_truth, [model_answer_before])
metrics_after = evaluate_all(model_answer_after, ground_truth, generated_attempts)

In [39]:
# ===================== DISPLAY METRICS =====================
# Exact Match (EM)
# Checks if the model's answer exactly matches the ground truth (case-insensitive).
# F1 (Token Overlap)
# Measures the harmonic mean of precision and recall based on shared tokens between prediction and ground truth.
# Accuracy (Overlap-based)
# Returns 1 if there's any token overlap between the prediction and ground truth, else 0.
# Pass@k
# Indicates if at least one out of k generated answers exactly matches the ground truth.

# METEOR
# Evaluates semantic similarity, considering synonyms, word order, and stemming, for paraphrased responses.
print("\n📊 Prompt Evaluation (Before Reranking):")
for k, v in metrics_before.items():
    print(f"{k}: {v}")

print("\n📊 Prompt Evaluation (After Reranking):")
for k, v in metrics_after.items():
    print(f"{k}: {v}")


📊 Prompt Evaluation (Before Reranking):
Exact Match (EM): 0
F1 (Token Overlap): 0.0675
Accuracy (Overlap-based): 1
Pass@3: 0
METEOR: 0.1401

📊 Prompt Evaluation (After Reranking):
Exact Match (EM): 0
F1 (Token Overlap): 0.0656
Accuracy (Overlap-based): 1
Pass@3: 0
METEOR: 0.1374


In [ ]:
# Display chunks after reranking
print("\nReranked Chunks (CrossEncoder):")
for i, (doc, score) in enumerate(sorted_docs):
    page = doc.metadata.get("page", "Unknown")
    print(f"\n--- Reranked Chunk {i+1} ---")
    print(f"Page: {page}")
    print(f"Rerank Score: {score:.4f}")
    print(f"Content:\n{doc.page_content[:300]}...")


🔸 Reranked Chunks (CrossEncoder):

--- Reranked Chunk 1 ---
Page: 18
Rerank Score: -10.3713
Content:
Aspects of a Class
Thursday, Feb 22nd 2024 19/22
The 5 aspects of the class are:
a
responsibility towards parent
interface towards callers
interface towards callees
responsibility towards inheritors
class'
purpose
a
Mike Lindner: The Five Principles For SOLID Software Design...

--- Reranked Chunk 2 ---
Page: 1
Rerank Score: -10.9173
Content:
Motivation
Thursday, Feb 22nd 2024 2/22
Find guiding design principles to
maintain software quality over
time....

--- Reranked Chunk 3 ---
Page: 12
Rerank Score: -10.9392
Content:
Liskov-Substitution - Contracts
Thursday, Feb 22nd 2024 13/22
“The Liskov Substitution Principle states, among other constraints,
that a subtype is not substitutable for its super type if it
strengthens its operations’ preconditions, or weakens its operations’
postconditions”
a
precondition
precondi...

--- Reranked Chunk 4 ---
Page: 16
Rerank Score: -11.3172
Content:
D

In [ ]:
# ===================== FINAL ANSWERS =====================

# Step 1: Answer using pre-reranked chunks
context_before = "\n\n".join([doc.page_content for doc in retrieved_docs[:3]])
prompt_before = prompt_template.format(context=context_before, question=question)
answer_before = llm.invoke(prompt_before)

# Step 2: Answer using top reranked chunks
top_reranked_docs = [doc for doc, _ in sorted_docs[:3]]
context_after = "\n\n".join([doc.page_content for doc in top_reranked_docs])
prompt_after = prompt_template.format(context=context_after, question=question)
answer_after = llm.invoke(prompt_after)

In [ ]:
# ===================== DISPLAY RESULTS =====================
display(Markdown("### Final Answer (Before Reranking):"))
display(Markdown(answer_before.content)) # Extract the text content

display(Markdown("### Final Answer (After Reranking):"))
display(Markdown(answer_after.content)) # Extract the text content

### Final Answer (Before Reranking):

The main objective of the document appears to be finding guiding design principles to maintain software quality over time, specifically focusing on the SOLID principles of software design and exploring aspects of a class.

### Final Answer (After Reranking):

The main objective of the document is to discuss guiding design principles, specifically the SOLID software design principles, to maintain software quality over time.